In [1]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output
print(check_output(["ls", "./input"]).decode("utf8"))

4convlayers_0.1731.csv
4L_pseudolabeling_0.1763.csv
decode_VGG0.1905.csv
sub201801102212renset50_0.17478.csv
sub201801111028inceptionv3_0.1869.csv
sub201801111144_xception_0.1841.csv
sub201801111534ResNet50_0.1689.csv
VGG16pretrain_0.1613.csv



In [2]:
sub_path = "./input"
all_files = os.listdir(sub_path)

# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_6,is_iceberg_7
0,5941774d,0.165616,0.040880,0.003044,1.483769e-05,0.106100,1.758107e-01,0.121406,0.035234
1,4023181e,0.440642,0.239020,0.990582,6.918396e-05,0.302233,4.762523e-01,0.207050,0.248752
2,b20200e4,0.157010,0.000118,0.006188,7.721067e-12,0.000325,6.421022e-07,0.005000,0.008674
3,e7f018bb,0.998759,0.948407,0.999728,9.999982e-01,0.995213,9.970161e-01,0.974939,0.988385
4,4371c8c3,0.199500,0.028982,0.998925,1.825918e-06,0.045377,2.452419e-02,0.066890,0.365645


In [3]:
# check correlation
concat_sub.corr()

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_6,is_iceberg_7
is_iceberg_0,1.000000,0.938561,0.622708,0.813792,0.945130,0.908958,0.915599,0.920000
is_iceberg_1,0.938561,1.000000,0.578125,0.833962,0.978560,0.932555,0.934714,0.937474
is_iceberg_2,0.622708,0.578125,1.000000,0.573847,0.584820,0.610635,0.599719,0.607147
is_iceberg_3,0.813792,0.833962,0.573847,1.000000,0.839452,0.820762,0.809369,0.797363
is_iceberg_4,0.945130,0.978560,0.584820,0.839452,1.000000,0.938131,0.954168,0.935658
is_iceberg_5,0.908958,0.932555,0.610635,0.820762,0.938131,1.000000,0.915116,0.900926
is_iceberg_6,0.915599,0.934714,0.599719,0.809369,0.954168,0.915116,1.000000,0.913580
is_iceberg_7,0.920000,0.937474,0.607147,0.797363,0.935658,0.900926,0.913580,1.000000


In [4]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:8].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:8].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:8].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:8].median(axis=1)

In [5]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.75
cutoff_hi = 0.25

# Mean Stacking LB

In [7]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_mean']
concat_sub[['id', 'is_iceberg']].to_csv('./sub2/stack_mean_2.csv', 
                                        index=False, float_format='%.6f')

# Median Stacking LB

In [8]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_median']
concat_sub[['id', 'is_iceberg']].to_csv('./sub2/stack_median_2.csv', 
                                        index=False, float_format='%.6f')

# PushOut + Median Stacking LB 0.1549

In [9]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:8] > cutoff_lo, axis=1), 0.9999, 
                                    np.where(np.all(concat_sub.iloc[:,1:8] < cutoff_hi, axis=1),
                                             0.0001, concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('./sub2/stack_pushout_median_2.csv', 
                                        index=False, float_format='%.6f')

# MinMax + Mean Stacking LB 0.1399

In [15]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:8] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:8] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_mean']))
concat_sub[['id', 'is_iceberg']].to_csv('./sub2/stack_minmax_mean_2.csv', 
                                        index=False, float_format='%.6f')

# MinMax + Median Stacking LB 0.1389

In [11]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:8] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:8] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('./sub2/stack_minmax_median_2.csv', 
                                        index=False, float_format='%.6f')

In [12]:
concat_sub

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_6,is_iceberg_7,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median,is_iceberg
0,5941774d,0.165616,0.040880,3.044423e-03,1.483769e-05,0.106100,1.758107e-01,0.121406,0.035234,0.175811,1.483769e-05,0.087553,0.106100,1.483769e-05
1,4023181e,0.440642,0.239020,9.905820e-01,6.918396e-05,0.302233,4.762523e-01,0.207050,0.248752,0.990582,6.918396e-05,0.379407,0.302233,3.022327e-01
2,b20200e4,0.157010,0.000118,6.188220e-03,7.721067e-12,0.000325,6.421022e-07,0.005000,0.008674,0.157010,7.721067e-12,0.024092,0.000325,7.721067e-12
3,e7f018bb,0.998759,0.948407,9.997279e-01,9.999982e-01,0.995213,9.970161e-01,0.974939,0.988385,0.999998,9.484074e-01,0.987723,0.997016,9.999982e-01
4,4371c8c3,0.199500,0.028982,9.989253e-01,1.825918e-06,0.045377,2.452419e-02,0.066890,0.365645,0.998925,1.825918e-06,0.194886,0.045377,4.537696e-02
5,a8d9b1fd,0.426266,0.287695,6.959881e-01,2.394062e-04,0.356264,3.649666e-01,0.133505,0.193277,0.695988,2.394062e-04,0.323561,0.356264,3.562641e-01
6,29e7727e,0.036051,0.095649,2.132297e-02,4.904675e-03,0.055109,1.054916e-02,0.138971,0.050590,0.138971,4.904675e-03,0.051794,0.036051,4.904675e-03
7,92a51ffb,0.999701,0.996734,9.999089e-01,9.999994e-01,0.999192,9.999193e-01,0.994644,0.931697,0.999999,9.946438e-01,0.998585,0.999701,9.999994e-01
8,c769ac97,0.013392,0.002215,4.096438e-07,5.920749e-12,0.014380,6.406796e-07,0.005000,0.003342,0.014380,5.920749e-12,0.004998,0.002215,5.920749e-12
9,aee0547d,0.000115,0.001901,1.208634e-05,1.399700e-05,0.000606,4.235656e-05,0.005000,0.009724,0.005000,1.208634e-05,0.001099,0.000115,1.208634e-05


# MinMax + BestBase Stacking LB 0.1537

In [13]:
# load the model with best base performance
sub_base = pd.read_csv('./input/VGG16pretrain_0.1613.csv')

In [14]:
concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:8] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:8] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_base']))
concat_sub[['id', 'is_iceberg']].to_csv('./sub2/stack_minmax_bestbase_2.csv', 
                                        index=False, float_format='%.6f')